<a href="https://colab.research.google.com/github/Nukaraju2003/Natural-Language-Processing/blob/main/Eng_to_telu_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
# Import required libraries
import warnings
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, MarianMTModel, MarianTokenizer
warnings.filterwarnings('ignore')

In [ ]:
# Translation models and tokenizers setup
nllb_model_name = "facebook/nllb-200-distilled-1.3B"
nllb_tokenizer = AutoTokenizer.from_pretrained(nllb_model_name)
nllb_model = AutoModelForSeq2SeqLM.from_pretrained(nllb_model_name)

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
# Helper function to split long text into chunks
def split_text_into_chunks(text, max_tokens, tokenizer):
    tokens = tokenizer.encode(text)
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk_tokens = tokens[i:i + max_tokens]
        chunks.append(tokenizer.decode(chunk_tokens, skip_special_tokens=True))
    return chunks

In [ ]:
# Helper function to translate chunks of text
def translate_chunks(chunks, translation_function):
    translated_chunks = [translation_function(chunk) for chunk in chunks]
    return " ".join(translated_chunks)

In [ ]:
# Wrapper function for large text translation
def translate_large_text(text, translation_function, tokenizer, max_tokens=512):
    chunks = split_text_into_chunks(text, max_tokens, tokenizer)
    translated_text = translate_chunks(chunks, translation_function)
    return translated_text

In [ ]:
def english_to_telugu(text):
    src_lang = "eng_Latn"
    tgt_lang = "tel_Telu"
    nllb_tokenizer.src_lang = src_lang
    inputs = nllb_tokenizer(text, return_tensors="pt", padding=True)
    outputs = nllb_model.generate(**inputs, forced_bos_token_id=nllb_tokenizer.convert_tokens_to_ids(tgt_lang))
    return nllb_tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
def translate(translate_large_text, tokenizer, target_lang_code='tel_Telu'):
    text = "Hello, I am raju"
    print("Transcription Complete!")
    print("Transcribed Text:", text)

    # Step 2: Translate the text
    translated_text = translate_large_text(text, english_to_telugu, tokenizer)
    print("Translated Text:")
    print(translated_text)

    return translated_text

In [ ]:
# Run the pipeline
translated_text = translate(translate_large_text, nllb_tokenizer)

Transcription Complete!
Transcribed Text: Hello, I am raju
Translated Text:
హలో, నేను రాజూ.


#MBart

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# Set the source language to English
tokenizer.src_lang = "en_XX"

# Input English text
english_text = "Hello, how are you?"

# Tokenize the input
encoded_en = tokenizer(english_text, return_tensors="pt")

# Set the target language to Telugu
forced_bos_token_id = tokenizer.lang_code_to_id["te_IN"]

# Generate the translation
generated_tokens = model.generate(
    **encoded_en,
    forced_bos_token_id=forced_bos_token_id
)

# Decode the generated tokens to get the Telugu translation
telugu_translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

print(telugu_translation)

['హలో, ఎలా ఉన్నావు?']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import os

model_dir = "/content/drive/MyDrive/My Projects/Trizen/GAPT1VST/mbart_model"

def load_model(model_dir):
    if not os.path.exists(model_dir):
        model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
        tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
        model.save_pretrained(model_dir)
        tokenizer.save_pretrained(model_dir)
        return model, tokenizer
    else:
        return MBartForConditionalGeneration.from_pretrained(model_dir), MBart50TokenizerFast.from_pretrained(model_dir)

def translate_text(model, tokenizer, text, src_lang, tgt_lang):
    if text is not None and isinstance(text, str):
        if tokenizer is None:
            print("Tokenizer is not loaded correctly")
            return None
        else:
            encoded_text = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
            tokenizer.src_lang = src_lang
            forced_bos_token_id = tokenizer.lang_code_to_id[tgt_lang]
            generated_tokens = model.generate(**encoded_text, forced_bos_token_id=forced_bos_token_id)
            translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            return translation
    else:
        print("Input text is invalid!")
        return None

model, tokenizer = load_model(model_dir)
# tokenizer.src_lang = "en_XX"
english_text = "Hello, how are you?"
src_lang = "en_XX"
tgt_lang = "te_IN"

telugu_translation = translate_text(model, tokenizer, english_text, src_lang, tgt_lang)
print(telugu_translation)




/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


['హలో, ఎలా ఉన్నావు?']
